# Text generation with an RNN

Aquest tutorial demostra com generar text utilitzant una RNN basada en caràcters. Treballaràs amb un conjunt de dades d'escriptura de Shakespeare. Donada una seqüència de caràcters d'aquestes dades ("Shakespear"), entrenarem un model per predir el següent caràcter de la seqüència ("e"). Es poden generar seqüències de text més llargues cridant el model repetidament.


<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import os
import time
import requests

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Descarrega el *dataset*

El *dataset* el podem trobar a [l'enllaç](https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt)

In [4]:
!wget 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

--2025-12-09 15:49:37--  https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.200.91, 216.58.215.187, 142.250.184.187, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.200.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt.1’

shakespeare.txt.1   100%[===================>]   1.06M  1.82MB/s    in 0.6s    

2025-12-09 15:49:38 (1.82 MB/s) - ‘shakespeare.txt.1’ saved [1115394/1115394]



### Llegim les dades


In [5]:
text = open("shakespeare.txt", 'rb').read().decode(encoding='utf-8')

print(f'Longitud del text: {len(text)} caràcters')

Longitud del text: 1115394 caràcters


In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(set(text))
print(f'{len(vocab)} caràcters únics')

65 caràcters únics


## Processament del text

### Vectorització del text

Abans de l'entrenament, cal convertir les cadenes a una representació numèrica. A PyTorch, sovint creem diccionaris senzills per *mapejar* caràcters a índexs i viceversa.

In [8]:
# Mapeig de caràcters a enters
char2idx = # TODO
# Mapeig d'enters a caràcters
idx2char = # TODO 

# Convertim tot el text a enters
text_as_int = # TODO

print(f"Vocabulari: {len(vocab)}")
print(f"Mostra mapeig: 'a' -> {char2idx['a']}")

Vocabulari: 65
Mostra mapeig: 'a' -> 39


In [9]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

class ShakespeareDataset(Dataset):
    def __init__(self, text_as_int, seq_length):
        self.text_as_int = text_as_int
        self.seq_length = seq_length
        self.total_sequences = len(text_as_int) // (seq_length + 1)

    def __len__(self):
        return self.total_sequences

    def __getitem__(self, idx):
        # Índex d'inici per a la seqüència actual
        start = idx * (self.seq_length + 1)
        # Agafem seq_length + 1 caràcters
        chunk = self.text_as_int[start : start + self.seq_length + 1]
        
        # Entrada: caràcters 0 fins al penúltim
        input_seq = # TODO
        # Objectiu: caràcters 1 fins l'últim
        target_seq = # TODO
        
        return input_seq, target_seq

# Crear el dataset i el dataloader
dataset = ShakespeareDataset(text_as_int, seq_length)
BATCH_SIZE = 64
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

## `nn.Embedding`

Explicació a partir d'[Stackoverflow](https://stackoverflow.com/questions/75646273/what-is-the-difference-nn-embedding-and-nn-linear) i emprant Gemini.


Els models de xarxes neuronals operen mitjançant operacions matemàtiques sobre nombres reals. No obstant això, el text és intrínsecament discret (caràcters o paraules). Tradicionalment, s'utilitzava la codificació **One-Hot**, on cada element del vocabulari es representa com un vector de zeros amb un únic '1' en la posició corresponent al seu índex.

Aquesta aproximació presenta dos problemes crítics:
* Ineficiència Dimensional: Per a un vocabulari de mida $∣V∣$, es requereixen vectors de dimensió $∣V∣$. En vocabularis grans (ex: 50.000 paraules), això genera vectors extremadament *esparsos* i computacionalment costosos.

* Manca de Semàntica: En una representació **One-Hot**, tots els vectors són ortogonals entre si (el producte escalar és 0). Això impedeix que el model capturi relacions de similitud; per a la xarxa, "ca" i "moix" són tan diferents com "ca" i "taula".

La capa __nn.Embedding__ resol aquests problemes projectant els índexs discrets en un espai vectorial continu de dimensió reduïda (anomenat $d_{model}$ o `embedding_dim`).

La següent infografia resumeix el procés que succeeix cada vegada que passem una lletra a aquesta capa:

![](infografia.png)

#### Explicació pas a pas:

1.  **L'Entrada (Índex Discret):**
    * El model rep un nombre enter (per exemple, l'índex `3` que representa la lletra `'d'`). 
    * Aquest nombre no té cap significat matemàtic per si sol (el 3 no és "més" que el 2).

2.  **La "Lookup Table" (La Matriu $E$):**
    * La capa d'Embedding és, en essència, una matriu de dimensions `(vocab_size, embedding_dim)`.
    * En el nostre cas, és una matriu de `65 x 256`.
    * **Important:** Aquesta matriu conté **pesos entrenables**. Al principi són aleatoris, però la xarxa els anirà modificant (aprenent) durant l'entrenament.

3.  **L'Operació (Selecció):**
    * En lloc de fer càlculs complexos, la xarxa utilitza l'índex d'entrada per **seleccionar i extreure** directament la fila corresponent.
    * Si l'entrada és `3`, copiem la fila 3.

4.  **La Sortida (Vector Dens):**
    * El resultat és un vector de nombres decimals (mida 256) que ara sí que conté informació "rica" sobre el caràcter.
    * Aquest vector és el que alimentem a la capa recurrent.

In [38]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # batch_first=True perquè les nostres entrades tenen forma (batch, seq, feature)
        self.rnn = nn.RNN(embedding_dim, rnn_units, batch_first=True)
        
        self.dense = nn.Linear(rnn_units, vocab_size)

    def forward(self, x, hidden=None):
        # x shape: (batch_size, seq_length)
        x = self.embedding(x)
        
        # output shape: (batch_size, seq_length, rnn_units)
        # hidden shape: (1, batch_size, rnn_units) -> estat final
        output, hidden = self.rnn(x, hidden)
        
        # Passem la sortida per la capa densa per obtenir logits
        output = self.dense(output)
        
        return output, hidden
    
    def init_hidden(self, batch_size):
        # Inicialitza l'estat ocult a zero
        return torch.zeros(1, batch_size, rnn_units).to(device)

Instancia el model

In [ ]:
# Paràmetres del model
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = RNNModel(vocab_size, embedding_dim, rnn_units).to(device)
print(model)

# Avaluació

In [ ]:
# Agafem un batch del dataloader
input_example_batch, target_example_batch = next(iter(dataloader))
input_example_batch = input_example_batch.to(device)

# Pas endavant inicial
example_batch_predictions, _ = model(input_example_batch)
print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [41]:
# Provem-ho per al primer exemple del batch
sampled_indices = torch.distributions.Categorical(logits=example_batch_predictions[0]).sample()
sampled_indices = sampled_indices.cpu().numpy()

print("Entrada: \n", "".join(idx2char[input_example_batch[0].cpu().numpy()]))
print()
print("Prediccions del següent caràcter: \n", "".join(idx2char[sampled_indices]))

Entrada: 
 ls true, 'tis there,
That, like an eagle in a dove-cote, I
Flutter'd your Volscians in Corioli:
Alon

Prediccions del següent caràcter: 
 YBjbm LHL.lwfolSyXDbT$vTk!BgTWDQuRoy3Cmz 'K'pp!zeRpzxgfFLL-BPXW&. xRQTCx$sJ.QkHElRcsycoRAy?eE?-3B.uy


In [42]:
loss_fn = # TODO
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Entrenament

In [43]:
EPOCHS = 20 # Pots augmentar-ho per millors resultats
print_every = 50

# Historial de pèrdues
history = []

model.train() # Mode entrenament

for epoch in range(EPOCHS):
    start = time.time()
    epoch_loss = 0
    hidden = None # Reiniciem l'estat ocult a l'inici de cada època (opcional, depèn de l'estratègia)
    
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Reiniciar gradients
        optimizer.zero_grad()
        
        # Forward pass
        output, _ = model(inputs)
        
        # Flatten per calcular la pèrdua
        # Output: (batch * seq_len, vocab_size)
        # Targets: (batch * seq_len)
        loss = loss_fn(output.reshape(-1, vocab_size), targets.reshape(-1))
        
        # Backward pass
        loss.backward()
        
        # Actualitzar pesos
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if batch_idx % print_every == 0:
            print(f"Època {epoch+1} Lot {batch_idx} Pèrdua {loss.item():.4f}")

    print(f'Temps per a l\'època {epoch+1}: {time.time() - start:.2f} seg')
    print(f'Pèrdua mitjana de l\'època: {epoch_loss / len(dataloader):.4f}')
    print('_'*80)

Època 1 Lot 0 Pèrdua 4.1844
Època 1 Lot 50 Pèrdua 2.0527
Època 1 Lot 100 Pèrdua 1.9310
Època 1 Lot 150 Pèrdua 1.7340
Temps per a l'època 1: 5.49 seg
Pèrdua mitjana de l'època: 2.0146
________________________________________________________________________________
Època 2 Lot 0 Pèrdua 1.7000
Època 2 Lot 50 Pèrdua 1.6490
Època 2 Lot 100 Pèrdua 1.6036
Època 2 Lot 150 Pèrdua 1.5357
Temps per a l'època 2: 5.52 seg
Pèrdua mitjana de l'època: 1.6189
________________________________________________________________________________
Època 3 Lot 0 Pèrdua 1.5047
Època 3 Lot 50 Pèrdua 1.5232
Època 3 Lot 100 Pèrdua 1.5288
Època 3 Lot 150 Pèrdua 1.4592
Temps per a l'època 3: 5.53 seg
Pèrdua mitjana de l'època: 1.5029
________________________________________________________________________________
Època 4 Lot 0 Pèrdua 1.4392
Època 4 Lot 50 Pèrdua 1.4595
Època 4 Lot 100 Pèrdua 1.4556
Època 4 Lot 150 Pèrdua 1.4190
Temps per a l'època 4: 5.50 seg
Pèrdua mitjana de l'època: 1.4409
_________________________

In [45]:
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    # Mode avaluació (important per dropout o batchnorm si n'hi hagués)
    model.eval()

    # Convertir cadena inicial a números (vectoritzar)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = torch.tensor(input_eval, dtype=torch.long).unsqueeze(0).to(device)

    # Emmagatzemar el text generat
    text_generated = []

    # Inicialitzar estat ocult
    hidden = None

    with torch.no_grad():
        for i in range(num_generate):
            # Forward pass
            output, hidden = model(input_eval, hidden)

            # Només ens interessa l'últim pas de temps (l'últim caràcter predit)
            predictions = output[:, -1, :] 
            
            # Aplicar temperatura
            predictions = predictions / temperature
            
            # Mostrejar utilitzant una distribució categòrica
            dist = torch.distributions.Categorical(logits=predictions)
            predicted_id = dist.sample()

            # Passar el caràcter predit com a següent entrada
            # Afegim una dimensió de batch (1, 1)
            input_eval = predicted_id.unsqueeze(0)

            # Convertir a caràcter i guardar
            text_generated.append(idx2char[predicted_id.item()])

    return (start_string + ''.join(text_generated))

# Feines a fer

1. Completar el codi.
2. Provar d'entrenar el model amb el text `tirant.txt` (trobau-lo a la mateixa carpeta de `GitHub`)